In [1]:
import pickle
import numpy as np
#import tensorflow as tf
from sklearn.preprocessing import scale

#from keras.models import Sequential
#from keras.optimizers import SGD, Adam, Adagrad
#from keras import backend as K
#from keras.layers import Embedding
#from keras.layers import Dense, Reshape, Concatenate, Activation, Dropout
#from keras.callbacks import ModelCheckpoint

import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, TensorDataset

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
train_cache = 'train2.pickle'
train_labels_cache = 'train-labels2.npy'
validation_cache = 'validation2.pickle'
validation_labels_cache = 'validation-labels2.npy'
test_cache = 'test2.pickle'
test_labels_cache = 'test-labels2.npy'
competition_test_cache = 'competition-test2.pickle'
metadata_cache = 'metadata2.pickle'

In [4]:
train = pd.read_pickle(train_cache)
validation = pd.read_pickle(validation_cache)
test = pd.read_pickle(test_cache)

train_labels = np.load(train_labels_cache)
validation_labels = np.load(validation_labels_cache)
test_labels = np.load(test_labels_cache)

competition_test = pd.read_pickle(competition_test_cache)
with open(metadata_cache, 'rb') as handle:
    metadata = pickle.load(handle)

In [5]:
def process_features(df):
    return torch.tensor(df[['QUARTER_HOUR','DAY_OF_WEEK','WEEK_OF_YEAR','ORIGIN_CALL_ENCODED','TAXI_ID_ENCODED','ORIGIN_STAND_ENCODED',
                           'dt_2014-08-14 00:00:00','dt_2014-09-30 00:00:00','dt_2014-10-06 00:00:00','dt_2014-11-01 00:00:00',
                           'dt_2014-12-21 00:00:00']].values)
# def process_features(df):
#    return torch.tensor(df[['QUARTER_HOUR','DAY_OF_WEEK','WEEK_OF_YEAR','ORIGIN_CALL_ENCODED','TAXI_ID_ENCODED',
#                            'ORIGIN_STAND_ENCODED']].values)
#def process_features(df):
#    return torch.tensor(df[['QUARTER_HOUR','DAY_OF_WEEK','WEEK_OF_YEAR','ORIGIN_STAND_ENCODED','dt_2014-08-14 00:00:00','dt_2014-09-30 00:00:00','dt_2014-10-06 00:00:00','dt_2014-11-01 00:00:00',
#                           'dt_2014-12-21 00:00:00']].values)

In [6]:
class LSTMNet(nn.Module):
    def __init__(self):
        super(LSTMNet, self).__init__()
        self.embed_quarter_hour = nn.Embedding(metadata['n_quarter_hours'], 10)
        self.embed_day_of_week = nn.Embedding(metadata['n_days_per_week'], 10)
        self.embed_week_of_year = nn.Embedding(metadata['n_weeks_per_year'],10)
        self.embed_client_ids = nn.Embedding(metadata['n_client_ids'],10)
        self.embed_taxi_ids = nn.Embedding(metadata['n_taxi_ids'],10)
        self.embed_stand_ids = nn.Embedding(metadata['n_stand_ids'],10)

        self.lstm = nn.LSTM(input_size=65, hidden_size=180, num_layers=5, batch_first=True, dropout=0.5)
        self.fc = nn.Linear(180, 1)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        qhr = self.embed_quarter_hour(x[:,0].to(torch.int32))
        dow = self.embed_day_of_week(x[:,1].to(torch.int32))
        woy = self.embed_week_of_year(x[:,2].to(torch.int32))
        ci = self.embed_client_ids(x[:,3].to(torch.int32))
        ti = self.embed_taxi_ids(x[:,4].to(torch.int32))
        si = self.embed_stand_ids(x[:,5].to(torch.int32))
        x = torch.cat([qhr,dow,woy,ci,ti,si,x[:,6:]],axis=1)
        x = x.to(torch.float32)

        # Reshape input data to (batch_size, sequence_length, input_size) for LSTM
        x = x.view(x.size(0), 1, -1)

        x, _ = self.lstm(x)
        x = self.relu(x)
        x = self.fc(x[:, -1, :])

        return x


In [7]:
import torch.optim as optim
learning_rate = 0.0001
criterion = nn.MSELoss()
num_epochs = 230

In [8]:
train_data = process_features(train)
train_data = TensorDataset(train_data, torch.tensor(train_labels))
validate_data = process_features(validation)
validate_data = TensorDataset(validate_data,torch.tensor(validation_labels))
BATCH_SIZE = 128
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(validate_data, batch_size = BATCH_SIZE,shuffle=True)
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
all_losses = []
train_loss = []
validation_losses = []

In [7]:
model = LSTMNet()
model = model.to(device)

In [37]:
print(model)

LSTMNet(
  (embed_quarter_hour): Embedding(96, 10)
  (embed_day_of_week): Embedding(7, 10)
  (embed_week_of_year): Embedding(52, 10)
  (embed_client_ids): Embedding(20778, 10)
  (embed_taxi_ids): Embedding(440, 10)
  (embed_stand_ids): Embedding(64, 10)
  (lstm): LSTM(65, 180, num_layers=5, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=180, out_features=1, bias=True)
  (relu): ReLU()
)


In [23]:
model.load_state_dict(torch.load('model_state_LSTM2\model_epoch199_LSTM_5Layer_Kaggle684.pth'))

<All keys matched successfully>

In [24]:
predict_input = torch.tensor(process_features(competition_test)).to(device)
print(predict_input.shape)

torch.Size([320, 11])


C:\Users\ikuto\AppData\Local\Temp\ipykernel_27168\1078090879.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  predict_input = torch.tensor(process_features(competition_test)).to(device)


In [25]:
def predict(model):
    predict_output = model(predict_input)
    return predict_output

In [31]:
# Delete PyTorch tensors
del train_loader 

# Free up memory and delete any cached objects
torch.cuda.empty_cache()

In [26]:
out = predict(model)

In [33]:
embed_mlp_predict = pd.read_csv('test_public.csv')
embed_mlp_predict = embed_mlp_predict['TRIP_ID']
predict_tensor = out.to('cpu').detach().numpy().flatten()
embed_mlp_predict= pd.concat([embed_mlp_predict, pd.DataFrame(predict_tensor)], axis=1)
embed_mlp_predict = embed_mlp_predict.rename(columns={0: 'TRAVEL_TIME'})
embed_mlp_predict[0:50]

,TRIP_ID,TRAVEL_TIME
0,T1,634.278198
1,T2,686.322083
2,T3,653.574158
3,T4,562.986023
4,T5,643.718079
5,T6,460.430847
6,T7,681.245300
7,T8,648.546936
8,T9,594.718567
9,T10,656.833191


In [36]:
embed_mlp_predict

,TRIP_ID,TRAVEL_TIME
0,T1,634.278198
1,T2,686.322083
2,T3,653.574158
3,T4,562.986023
4,T5,643.718079
...,...,...
315,T323,654.267883
316,T324,624.172729
317,T325,748.884033
318,T326,873.149597


In [34]:
embed_mlp_predict['TRAVEL_TIME'].mean()

820.4108

In [35]:
embed_mlp_predict.to_csv('Embedding_MLP_188_retrain.csv', index=False)